In [ ]:
import http
import requests

In [ ]:
class Virustotal:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://www.virustotal.com/vtapi/v2'
        self.collection = 'file'
        
    def __get_scan_data(self, file):
        
        function = 'scan'
        scan_params = {'apikey': self.api_key}
        files = {'file': (file, open(file, 'rb'))}
        url = f'{self.base_url}/{self.collection}/{function}'
        
        try:
            response = requests.post(url, files=files, params=scan_params)
            self.scan_status_code = response.status_code
            return response.json()
            
        except self.scan_status_code != 200:
            phrase = http.HTTPStatus(status_code).phrase
            print(f'{status_code}: {phrase}') 
            
    def __get_report_data(self, file):
        
        self.__set_scan_response(file)
        
        function = 'report'
        resource = self.scan['resource']
        params = {'apikey': self.api_key, 'resource': resource}
        url = f'{self.base_url}/{self.collection}/{function}'
        
        try:
            response = requests.get(url, params=params)
            self.report_status_code = response.status_code
            return response.json()
            
        except self.report_status_code != 200:
            phrase = http.HTTPStatus(status_code).phrase
            print(f'{status_code}: {phrase}')
    
    def __set_scan_response(self, file):
        self.scan = self.__get_scan_data(file)
            
    def __set_report_response(self, file):
        self.report = self.__get_report_data(file)
        
    def check_file(self, file):
        
        self.__set_report_response(file)
        
        report = self.report
        total = report['total']
        positives = report['positives']
        name = file.split('/')[-1]
        return f'the file - [{name}] tested positive from {positives} out of {total}'

In [ ]:
api = Virustotal('f0146bfb2784cbbd3e10efe57964ac8cbcbc58f103f631f7e1f360a8f986c4a0')
report_string = api.check_file('eicar_test_file.txt')

In [ ]:
import os
import tkinter as tk

window = tk.Tk()
window.title('Virus Detection')
# except a file name as a input
entry_instruction = tk.Label(
    window, 
    text='Enter File Name:', 
    font='Helvetica 16 bold'
) 
entry_instruction.grid(row=0, column=0, columnspan=1)

entry = tk.Entry(
    window, 
    font='Helvetica 16'
)
entry.grid(row=0, column=1, columnspan=1)

# submit the file with a button 
def submit():
    key = 'f0146bfb2784cbbd3e10efe57964ac8cbcbc58f103f631f7e1f360a8f986c4a0'
    file_name = entry.get()
    # find the files path in the computer
    cur_dir = list(os.walk(os.getcwd()))
    for path, _, files in cur_dir:
        for file in files:
            if file == file_name:
                api = Virustotal(key)
                report_string = api.check_file(f'{path}/{file_name}')
                
                report_label = tk.Label(
                    window,
                    text=report_string,
                    font='Helvetica 18 bold'
                )
                report_label.grid(row=1, column=0, columnspan=3)
                return
            
    not_found_label = tk.Label(
        window,
        text=f'the file - [{file_name}] was not found.',
        font='Helvetica 18 bold'
    )
    not_found_label.grid(row=1, column=0, columnspan=3)



submit_button = tk.Button(
    window,
    text='check',
    command=submit,
    font='Helvetica 16 bold'
)
submit_button.grid(row=0, column=2, columnspan=1)
window.mainloop()
